In [30]:
import torch
import numpy as np
from PIL import Image
import requests
from transformers import SamModel, SamProcessor


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [4]:
model = SamModel.from_pretrained("facebook/sam-vit-huge").to(device)
processor = SamProcessor.from_pretrained("facebook/sam-vit-huge")

/Users/vp/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/vp/anaconda3/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c106detail19maybe_wrap_dim_slowIxEET_S2_S2_b
  Referenced from: <E03EDA44-89AE-3115-9796-62BA9E0E2EDE> /Users/vp/anaconda3/lib/python3.11/site-packages/torchvision/image.so
  Expected in:     <F2FE5CF8-5B5B-3FAD-ADF8-C77D90F49FC9> /Users/vp/anaconda3/lib/python3.11/site-packages/torch/lib/libc10.dylib'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [136]:
raw_image = None

In [137]:
if not raw_image:
    raw_image = Image.open("example_1.jpg")
    input_points = [[[int(raw_image.width / 2), int(raw_image.height / 2)]]]
    
input_points

[[[1863, 1045]]]

In [138]:
if not raw_image:
    img_url = "https://huggingface.co/ybelkada/segment-anything/resolve/main/assets/car.png"
    raw_image = Image.open(requests.get(img_url, stream=True).raw).convert("RGB")
    input_points = [[[450, 600]]]  # 2D location of a window in the image

In [139]:
inputs = processor(raw_image, input_points=input_points,
                   return_tensors="pt").to(device)
outputs = model(**inputs)

masks = processor.image_processor.post_process_masks(
    outputs.pred_masks.cpu(), inputs["original_sizes"].cpu(
    ), inputs["reshaped_input_sizes"].cpu()
)
scores = outputs.iou_scores

In [140]:
scores

tensor([[[0.9748, 0.9585, 0.9598]]], grad_fn=<SliceBackward0>)

In [145]:
def show_mask(mask, filename, random_color=True):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    mask_image = mask_image * 255
    np_array = mask_image.cpu().numpy()  # Convert to NumPy array
    # Create a PIL Image from the NumPy array
    image = Image.fromarray(np.uint8(np_array))
    return image

def get_mask_image(masks, scores, raw_image):
    if len(masks.shape) == 4:
        masks = masks.squeeze()
    if scores.shape[0] == 1:
      scores = scores.squeeze()
    nb_predictions = scores.shape[-1]
    for i, (mask, score) in enumerate(zip(masks, scores)):
        image = show_mask(mask, f"output_mask_{i}.png")
        raw_image.paste(image, (0, 0), image)
    raw_image.save("output.png")

get_mask_image(masks[0], scores, raw_image)

In [106]:
# Assuming you have a PyTorch tensor of shape [1764, 2646, 4] with values in the range [0, 1]
# You can create a NumPy array from it and map the values to [0, 255].
pytorch_tensor = torch.randn(1764, 2646, 4)  # Replace this with your actual data
scaled_tensor = (pytorch_tensor * 255).clamp(0, 255)  # Scale values to [0, 255] and clamp

# Convert to NumPy array with uint8 data type
numpy_array = scaled_tensor.cpu().numpy().astype(np.uint8)

# Create a PIL Image from the NumPy array
image = Image.fromarray(numpy_array)

# Save the image to a PNG file
image.save('output.png')
